In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# Parámetros de búsqueda
title = "analista de datos"
location = "United States"

# URL base para la paginación
base_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
id_list = []
fecha_actual = datetime.now()

# Lógica para paginar y obtener más IDs
for start in range(0, 200, 25):  # Cambiar 200 según el número máximo deseado
    list_url = f"{base_url}?keywords={title.replace(' ', '%2B')}&location={location}&start={start}"
    response = requests.get(list_url)
    if response.status_code != 200:
        print(f"Error al obtener datos de la página con inicio {start}")
        break
    list_data = response.text
    list_soup = BeautifulSoup(list_data, "html.parser")
    page_jobs = list_soup.find_all("li")

    # Extraer IDs de los trabajos
    for job in page_jobs:
        try:
            base_card_div = job.find("div", {"class": "base-card"})
            if base_card_div:
                job_id = base_card_div.get("data-entity-urn").split(":")[3]
                id_list.append(job_id)
        except AttributeError:
            continue

# Limitar el número de IDs a procesar si es necesario
id_list = list(set(id_list))  # Eliminar duplicados
print(f"Se encontraron {len(id_list)} IDs únicos.")

# Lista para almacenar información de trabajos
job_list = []

# Obtener información de cada trabajo
for job_id in id_list:
    # URLs de la vacante
    job_url_api = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    job_url_decent = f"https://www.linkedin.com/jobs/view/{job_id}"  # URL limpia

    # Solicitar datos del trabajo
    job_response = requests.get(job_url_api)
    if job_response.status_code != 200:
        continue  # Saltar si hay error en la solicitud

    # Parsear datos del trabajo
    job_soup = BeautifulSoup(job_response.text, "html.parser")
    job_post = {
        "Estado": "Postulado",
        "Posición": None,
        "Fecha de Postulación": fecha_actual.strftime("%Y-%m-%d"),
        "Plataforma": "LinkedIn",
        "Empresa": None,
        "Rubro": None,
        "Nombre del Reclutador": None,
        "Link Posición": job_url_decent,
        "Link Empresa": None,
        "Link Reclutador": None,
        "Publicación": None,
        "Antigüedad (horas)": None,
        "Experiencia": None,
    }

    # Extraer detalles del trabajo
# try extract title position
    try:
        job_post["Posición"] = job_soup.find("h2", {"class": "topcard__title"}).text.strip()
    except AttributeError:
        pass
#try extract name link empresa
    try:
        company_tag = job_soup.find("a", {"class": "topcard__org-name-link"})
        job_post["Empresa"] = company_tag.text.strip()
        job_post["Link Empresa"] = company_tag.get("href")
    except AttributeError:
        pass
#try extract rubro empresa
    try:
        criterios = job_soup.find_all("span", {"class": "description__job-criteria-text"})
        if len(criterios) > 0:
            job_post["Rubro"] = criterios[3].text.strip()
    except AttributeError:
        pass
#try extract name and link recruiter
    try:
        recruiter_tag = job_soup.find("a", {"class": "base-card__full-link"})
        job_post["Nombre del Reclutador"] = recruiter_tag.text.strip()
        job_post["Link Reclutador"] = recruiter_tag.get("href")
    except AttributeError:
        pass

# try extract time publication
    try:
        publication_time = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"})
        job_post["Publicación"] = publication_time.text.strip() if publication_time else "No disponible"
    except AttributeError:
        job_post["Publicación"] = "No disponible"

    # Calcular antigüedad
    job_post["Antigüedad (horas)"] = (fecha_actual - datetime.strptime(job_post["Fecha de Postulación"], "%Y-%m-%d")).total_seconds() / 3600

    # Filtrar por antigüedad (menos de 168 horas) y más de 2 semanas
    if job_post["Antigüedad (horas)"] <= 168:  # 1 semana = 168 horas
        job_list.append(job_post)

# Ordenar trabajos por fecha de publicación y limitar a 100
job_list = sorted(job_list, key=lambda x: x["Fecha de Postulación"], reverse=True)[:100]

# Guardar los resultados en un archivo Excel
jobs_df = pd.DataFrame(job_list)
jobs_df.to_excel("LinkedIn_Recent_Jobs_Organized.xlsx", index=False, engine="openpyxl")

print("Se han guardado los resultados en 'LinkedIn_Recent_Jobs_4_quiality.xlsx'.")


Se encontraron 77 IDs únicos.
Se han guardado los resultados en 'LinkedIn_Recent_Jobs_4_quiality.xlsx'.


In [8]:
# Create a pandas DataFrame using the list of job dictionaries 'job_list'
jobs_df = pd.DataFrame(job_list)
jobs_df

,Estado,Posición,Fecha de Postulación,Plataforma,Empresa,Rubro,Nombre del Reclutador,Link Posición,Link Empresa,Link Reclutador,Publicación,Antigüedad (horas),Experiencia
0,Postulado,Data Analyst,2025-01-16,LinkedIn,Ascendion,"Technology, Information and Media",Mohammad Khatri,https://www.linkedin.com/jobs/view/4114865808,https://www.linkedin.com/company/ascendion?trk...,https://www.linkedin.com/in/mohammad-khatri-69...,1 week ago,19.98583,None
1,Postulado,PowerBI Data Analyst,2025-01-16,LinkedIn,US Tech Solutions,"Information Services, Banking, and Technology,...",Deepak Gupta,https://www.linkedin.com/jobs/view/4119902877,https://www.linkedin.com/company/us-tech-solut...,https://in.linkedin.com/in/deepak-gupta-3164272a,6 days ago,19.98583,None
2,Postulado,Data Analyst I,2025-01-16,LinkedIn,"Upgrade, Inc.",Financial Services,None,https://www.linkedin.com/jobs/view/4103369461,https://www.linkedin.com/company/upgrade-inc-?...,None,3 days ago,19.98583,None
3,Postulado,"Analyst, Data Science - Demand Planning Ops",2025-01-16,LinkedIn,Petco,Retail,None,https://www.linkedin.com/jobs/view/4127902487,https://www.linkedin.com/company/petco-animal-...,None,No disponible,19.98583,None
4,Postulado,Data Analyst,2025-01-16,LinkedIn,Peloton Interactive,"Technology, Information and Internet",None,https://www.linkedin.com/jobs/view/4118244865,https://www.linkedin.com/company/peloton-inter...,None,1 week ago,19.98583,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Postulado,Data Analyst,2025-01-16,LinkedIn,McAfee,Computer and Network Security,None,https://www.linkedin.com/jobs/view/4102473051,https://www.linkedin.com/company/mcafee?trk=pu...,None,1 month ago,19.98583,None
62,Postulado,SQL Data Analyst,2025-01-16,LinkedIn,INSPYR Solutions,Construction and Oil and Gas,None,https://www.linkedin.com/jobs/view/4122359378,https://www.linkedin.com/company/inspyrsolutio...,None,6 days ago,19.98583,None
63,Postulado,Data Analyst,2025-01-16,LinkedIn,Fitch Ratings,Financial Services,None,https://www.linkedin.com/jobs/view/4120748825,https://www.linkedin.com/company/fitch-ratings...,None,6 days ago,19.98583,None
64,Postulado,Data Analyst,2025-01-16,LinkedIn,Siemens,Automation Machinery Manufacturing,None,https://www.linkedin.com/jobs/view/4119558846,https://de.linkedin.com/company/siemens?trk=pu...,None,6 days ago,19.98583,None
